**مثال:** 

دو مشتری برتر یک سال مشخص

In [ ]:
SELECT TOP 2
	P.BusinessEntityID,
	P.FirstName,
	P.LastName,
	SUM(SOD.OrderQty * SOD.UnitPrice) AS TotalSales
FROM
	Sales.SalesOrderHeader AS SOH
INNER JOIN
	Sales.SalesOrderDetail AS SOD
ON	
	SOH.SalesOrderID = SOD.SalesOrderID
INNER JOIN
	Sales.Customer AS C
ON
	SOH.CustomerID = C.CustomerID
INNER JOIN
	Person.Person AS P
ON
	C.PersonID = P.BusinessEntityID
WHERE
	YEAR(SOH.OrderDate) = 2011
GROUP BY
	P.BusinessEntityID,
	P.FirstName,
	P.LastName
ORDER BY
	TotalSales DESC

**مثال:**

 دو مشتری برتر هر سال

-   **CROSS APPLY**

In [ ]:
SELECT 
	*
FROM
	Sales.vwSales_Years AS Y
CROSS APPLY
	(
		SELECT TOP 2
			P.BusinessEntityID,
			P.FirstName,
			P.LastName,
			SUM(SOD.OrderQty * SOD.UnitPrice) AS TotalSales
		FROM
			Sales.SalesOrderHeader AS SOH
		INNER JOIN
			Sales.SalesOrderDetail AS SOD
		ON	
			SOH.SalesOrderID = SOD.SalesOrderID
		INNER JOIN
			Sales.Customer AS C
		ON
			SOH.CustomerID = C.CustomerID
		INNER JOIN
			Person.Person AS P
		ON
			C.PersonID = P.BusinessEntityID
		WHERE
			YEAR(SOH.OrderDate) = Y.OrderYear
		GROUP BY
			P.BusinessEntityID,
			P.FirstName,
			P.LastName
		ORDER BY
			TotalSales DESC
	) AS TopCustomers
ORDER BY 
	Y.OrderYear

**TVF:**

دو مشتری برتر یک سال

پارامتر ورودی: سال

In [ ]:
DROP FUNCTION IF EXISTS Sales.tvfTopCustomers
GO

CREATE FUNCTION Sales.tvfTopCustomers
(
    @Year INT,
	@N INT
)
RETURNS TABLE AS RETURN
(
    SELECT TOP(@N)
		P.BusinessEntityID,
		P.FirstName,
		P.LastName,
		SUM(SOD.OrderQty * SOD.UnitPrice) AS TotalSales
	FROM
		Sales.SalesOrderHeader AS SOH
	INNER JOIN
		Sales.SalesOrderDetail AS SOD
	ON	
		SOH.SalesOrderID = SOD.SalesOrderID
	INNER JOIN
		Sales.Customer AS C
	ON
		SOH.CustomerID = C.CustomerID
	INNER JOIN
		Person.Person AS P
	ON
		C.PersonID = P.BusinessEntityID
	WHERE
		YEAR(SOH.OrderDate) = @Year
	GROUP BY
		P.BusinessEntityID,
		P.FirstName,
		P.LastName
	ORDER BY
		TotalSales DESC
)

**مثال:**

دو مشتری برتر هر سال

-   **CROSS APPLY**
-   **ّFunction**

In [ ]:
SELECT 
	*
FROM
	Sales.vwSales_Years AS Y
CROSS APPLY
	Sales.tvfTopCustomers(Y.OrderYear, 2) AS  TC
ORDER BY 
	Y.OrderYear

**مثال:**

 دو مشتری برتر هر سال

-   **OUTER APPLY**
-   **ّFunction**

In [ ]:
WITH Years
AS
(
			SELECT 2010 AS OrderYear
	UNION	SELECT 2011 AS OrderYear
	UNION	SELECT 2012 AS OrderYear
	UNION	SELECT 2013 AS OrderYear
	UNION	SELECT 2014 AS OrderYear
	UNION	SELECT 2015 AS OrderYear

)

SELECT 
	*
FROM
	Years AS Y
OUTER APPLY
	Sales.tvfTopCustomers(Y.OrderYear, 2) AS  TC
ORDER BY 
	Y.OrderYear